In [4]:
import numpy as np

In [5]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

In [128]:
def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(A):
    
    n = A.shape[0]
    M = np.copy(A)

    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    M[:,:] = M[:,permutations]
    M[:,:] = M[permutations,:]
    
    decomp, indices = two_level_decomp(M)
    new_decomp = []
    new_ind = []

    for i in range(len(indices)):
        
        t = np.take(permutations, indices[i])
        if t[0]>t[1]:
            new_decomp.append(decomp[i].T)
            new_ind.append(np.sort(t))

        else:
            new_decomp.append(decomp[i])
            new_ind.append(t)
            
    return new_decomp, new_ind

    

In [199]:
class SGate:
    def __init__(self,gate,Q_id):
        self.gate = gate
        self.Q_id = Q_id

        
class CGate:
    def __init__(self,gate,Q_id):
        self.gate = gate
        self.Q_id = Q_id

        
def flip(overlay, gates):
    
    '''X gate operator determined by XOR between qbit'''
    
    Q_id = 0
    while overlay>0:
        if overlay % 2 == 1:
            gates.append(SGate(Pauli_x, Q_id))
        overlay //= 2
        Q_id += 1

        
def SpecU_2_gate(A):
    
    '''Special Unitary to Gate'''
    
    u1 = A[0,0]
    u2 = A[0,1]
    theta = 2*np.arccos(abs(u1))
    lamda = np.angle(u1)
    mu = np.angle(u2)
    
    gates = []
    
    # U' = R_z (lamda+mu) * R_y (2*theta) * R_z (lamda-mu)
    
    # Single qbit can be implemented using 4 gates 
    # U = R_1 (phi) * R_z (lamda+mu) * R_y (theta) * R_z (lamda+mu)
    
    temp = lamda-mu
    if abs(temp) > thr:
        gates.append(np.diag([np.exp(0.5j * temp), np.exp(-0.5j * temp)]))
        
    if abs(theta) > thr:
        gates.append(np.array([[np.cos(theta/2), np.sin(theta/2)],
                               [-np.sin(theta/2), np.cos(theta/2)]]))
    
    temp = lamda+mu
    if abs(temp) > thr:
        gates.append(np.diag([np.exp(0.5j * temp), np.exp(-0.5j * temp)]))
    
    return gates
        
    
def U_2_gate(A):
    
    '''General Unitary to Gate'''
    
    phi = np.angle(np.linalg.det(A))
    if abs(phi) < thr:
        return SpecU_2_gate(A)
    elif np.allclose(A, Pauli_x):
        return Pauli_x
    else:
        A = np.diag([1.0, np.exp(-1j * phi)]) @ A
        return SpecU_2_gate(A) + np.diag([1.0, np.exp(1j * phi)])
    

def matrix_2_gates(A):
    
    '''Convert matrix into series of gates'''
    
    gates = []
    prev_flip_overlay = 0
    
    decomp, indices = gray_method(A)
    for i in range(len(decomp)):
        sub = decomp[i]
        print('sub: ',sub)
        print('')
        print('U_2 sub: ',U_2_gate(sub))
        ind1, ind2 = indices[i][0], indices[i][1]
        id_overlay = ind1 ^ ind2
        Q_id = int(np.log2(id_overlay))
        
        flip_overlay = sub.shape[0]-1 - ind2
        flip(flip_overlay ^ prev_flip_overlay, gates)
        
        for gate in U_2_gate(sub):
            gates.append(CGate(gate,Q_id))
#             print(i, gate, )
        prev_flip_overlay = flip_overlay
        
    flip(prev_flip_overlay,gates)
    
    return(gates)